# NEXT DATA MODEL (NDM)

## Definition of the NEXT (hdf5) DATA MODEL

In [1]:
from __future__ import print_function
from IPython.display import Image
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import tables as tb
import numpy as np

In [3]:
from Util import *
from PlotUtil import *
from PandaUtil import *

### NDM

#### Class NMD: represents the Next Data Model

In [ ]:
class NDM:
    import tables as tb
    """
    Represents the hdf5-based NEXT Data MODEL
    
    """   
    def __init__(self, path_to_file, access_mode='r'):
        """
        path_to_file is a full path to a hdf5 DST file
        the default access mode is r+ (read mode even if file not closed)
        
        
        """
        
        self.h5f =tb.open_file(path_to_file, access_mode) 
        
        self.geom_t = self.h5f.root.Detector.DetectorGeometry
        self.pmt_t = self.h5f.root.Sensors.DataPMT
        self.sipm_t = self.h5f.root.Sensors.DataSiPM
        self.mctrk_t = self.h5f.root.MC.MCTracks
        self.fee_t = self.h5f.root.MC.FEE
        
        self.epmt = self.h5f.root.RD.epmt
        self.esipm = self.h5f.root.RD.esipm
        self.pmtrd = self.h5f.root.pmtrd
        self.sipmrd = self.h5f.root.sipmrd
        self.pmtrwf = self.h5f.root.RD.pmtrwf
        self.sipmrwf = self.h5f.root.RD.sipmrwf
    
    def __del__(self):
        self.h5f.close()
        
    def Geometry(self):
        """
        returns a PD series for the geometry data
        """
        ga = self.geom_t.read()
        G = pd.Series([ga[0][0][0],ga[0][0][1],ga[0][1][0],ga[0][1][1],
                    ga[0][2][0],ga[0][2][1],ga[0][3]],
                    index=['xdet_min','xdet_max','ydet_min','ydet_max',
                            'zdet_min','zdet_max','R'])
        return G
    
    def FEE(self):
        """
        Reads the FEE table en returns a PD Series for the simulation parameters 
        and a PD series for the values of the capacitors used in the simulation
        """
        
        fa = self.fee_t.read()
        F = pd.Series([fa[0][0],fa[0][1],fa[0][2],fa[0][3],fa[0][5],fa[0][6],fa[0][7],fa[0][8],fa[0][9],fa[0][10],
                   fa[0][11],fa[0][12]],
                    index=['offset','pmt_gain','V_gain','R',"time_step",
                           "time_daq",
                            "freq_LPF",
                            "freq_HPF",
                            "LSB",
                            "volts_to_adc",
                            "noise_fee_rms",
                            "noise_adc"])
        C =pd.Series([fa[0][4]],index=['C12'])
        return F,C
    
    def get_column_(self,pmta,ic):
        """
        access column ic of table pmta and returns column as an array
        """
        col =[]
        for i in range(pmta.shape[0]):
            col.append(pmta[i][ic])
        return np.array(col)
 
    def Sensors(self,sensor='pmt'):
        """
        reads the sensors table and returns a data frame
        sensor can take the values 'pmt' or 'sipm'
        """
        pmta = eval('self.'+sensor+'_t').read()
        PMT={}
        PMT['channel'] = self.get_column_(pmta,0)
        PMT['active'] = self.get_column_(pmta,1)
        PMT['x'] = self.get_column_(pmta,2).T[0]
        PMT['y'] = self.get_column_(pmta,2).T[1]
        PMT['gain'] = self.get_column_(pmta,3)
        PMT['adc_to_pes'] = self.get_column_(pmta,4)
        
        return pd.DataFrame(PMT)
    
    def SensorsTrueEnergy(self,sensor='pmt'):
        """
        reads the sensors energy and returns a data frame:
        sensor can take the values, pmt or sipm
        """        
        return pd.DataFrame(eval('self.e'+sensor).read())
    
    def get_waveforms_(self,pmtea,event_number):
        """
        Takes the earray pmtea and returns a DF for event_number
        """
    
        PMTWF ={}
        NPMT = pmtea.shape[1]
    
        for j in range(NPMT):
            PMTWF[j] = pmtea[event_number, j] #waveform for event event_number, PMT j
       
        return pd.DataFrame(PMTWF)
    
    def GetMCRD(self,event_number=0):
        return self.get_waveforms_(self.pmtrd,event_number) 
    
    def GetMCRD_ZS(self,event_number=0):
        """
        Zero Suprressed WF
        """
        PMTWF ={}
        NPMT = pmtea.shape[1]
    
        for j in range(NPMT):
            PMTWF[j] = pmtea[event_number, j] #waveform for event event_number, PMT j
       
        return pd.DataFrame(PMTWF)
    
    def __str__(self): 
        return "{}".format(self.h5f.__str__)
        
    
    def __repr__(self): 
        return "{}".format(self.h5f.__repr__)
        
        

In [56]:
class NDM:
    import tables as tb
    """
    Represents the hdf5-based NEXT Data MODEL
    
    """   
    def __init__(self, path_to_file, access_mode='r+'):
        """
        path_to_file is a full path to a hdf5 DST file
        the default access mode is r+ (read mode even if file not closed)
        
        
        """
        
        self.h5f = tb.open_file(path_to_file) 
        self.geom_t = self.h5f.root.Detector.DetectorGeometry
        self.pmt_t = self.h5f.root.Sensors.DataPMT
        self.sipm_t = self.h5f.root.Sensors.DataSiPM
        self.mctrk_t = self.h5f.root.MC.MCTracks
        self.fee_t = self.h5f.root.MC.FEE
        
        self.epmt = self.h5f.root.RD.epmt
        self.esipm = self.h5f.root.RD.esipm
        self.pmtrd = self.h5f.root.RD.pmtrd
        self.sipmrd = self.h5f.root.RD.sipmrd
        
    def Geometry(self):
        """
        returns a PD series for the geometry data
        """
        ga = self.geom_t.read()
        G = pd.Series([ga[0][0][0],ga[0][0][1],ga[0][1][0],ga[0][1][1],
                    ga[0][2][0],ga[0][2][1],ga[0][3]],
                    index=['xdet_min','xdet_max','ydet_min','ydet_max',
                            'zdet_min','zdet_max','R'])
        return G
    
    def FEE(self):
        """
        Reads the FEE table en returns a PD Series for the simulation parameters 
        and a PD series for the values of the capacitors used in the simulation
        """
        
        fa = self.fee_t.read()
        F = pd.Series([fa[0][0],fa[0][1],fa[0][2],fa[0][3],fa[0][5],fa[0][6],fa[0][7],fa[0][8],fa[0][9],fa[0][10],
                   fa[0][11],fa[0][12]],
                    index=['offset','pmt_gain','V_gain','R',"time_step",
                           "time_daq",
                            "freq_LPF",
                            "freq_HPF",
                            "LSB",
                            "volts_to_adc",
                            "noise_fee_rms",
                            "noise_adc"])
        C =pd.Series([fa[0][4]],index=['C12'])
        return F,C
    
    def get_column_(pmta,ic):
        """
        access column ic of table pmta and returns column as an array
        """
        col =[]
        for i in range(pmta.shape[0]):
            col.append(pmta[i][ic])
        return np.array(col)
 
    def Sensors(self,sensor='pmt'):
        """
        reads the sensors table and returns a data frame
        sensor can take the values 'pmt' or 'sipm'
        """
        pmta = eval('self.'+sensor+'_t').read()
        PMT={}
        PMT['channel'] = get_column_(pmta,0)
        PMT['active'] = get_column_(pmta,1)
        PMT['x'] = get_column_(pmta,2).T[0]
        PMT['y'] = get_column_(pmta,2).T[1]
        PMT['gain'] = get_column_(pmta,3)
        PMT['adc_to_pes'] = get_column_(pmta,4)
        
        return pd.DataFrame(PMT)
    
    def SensorsTrueEnergy(self,sensor='pmt'):
        """
        reads the sensors energy and returns a data frame:
        sensor can take the values, pmt or sipm
        """        
        return pd.DataFrame(eval('self.e'+sensor).read())
    
    def get_waveforms(pmtea,event_number=0):
    """
    Takes the earray pmtea and returns a DF for event_number
    """
    
    PMTWF ={}
    NPMT = pmtea.shape[1]
    
    for j in range(NPMT):
        PMTWF[j] = pmtea[event_number, j] #waveform for event event_number, PMT j
       
    return pd.DataFrame(PMTWF)
        

In [57]:
ndm.h5f.close()

In [58]:
ndm = NDM(path_to_file='../../data/Waveforms/25ns/WF_Na_1k_RWF.h5', access_mode ="r+") 

In [51]:
ndm.Sensors(sensor='pmt')

,active,adc_to_pes,channel,gain,x,y
0,1,20.0,0,4500000.0,23.9414,65.7785
1,1,20.0,1,4500000.0,-68.9365,-12.1554
2,1,20.0,2,4500000.0,44.9951,-53.6231
3,1,20.0,3,4500000.0,0.0000,185.0000
4,1,20.0,5,4500000.0,-118.9160,141.7180
5,1,20.0,6,4500000.0,-182.1890,32.1249
6,1,20.0,8,4500000.0,-160.2150,-92.5000
7,1,20.0,9,4500000.0,-63.2737,-173.8430
8,1,20.0,10,4500000.0,63.2737,-173.8430
9,1,20.0,12,4500000.0,160.2150,-92.5000


In [53]:
ndm.Sensors(sensor='sipm')

,active,adc_to_pes,channel,gain,x,y
0,1,1.0,1000,1.0,235.0,45.0
1,1,1.0,1001,1.0,235.0,55.0
2,1,1.0,1002,1.0,235.0,65.0
3,1,1.0,1003,1.0,235.0,75.0
4,1,1.0,1004,1.0,235.0,85.0
5,1,1.0,1005,1.0,235.0,95.0
6,1,1.0,1006,1.0,235.0,105.0
7,1,1.0,1007,1.0,235.0,115.0
8,1,1.0,1008,1.0,225.0,45.0
9,1,1.0,1009,1.0,225.0,55.0


In [59]:
ndm.SensorsTrueEnergy(sensor='pmt')

,0,1,2,3,4,5,6,7,8,9,10,11
0,5925,5976,5905,5053,5061,5025,5107,5100,5053,5017,5089,5108
1,4475,4669,4645,3970,3943,3930,3896,4056,3954,4064,3890,3958
2,20056,20176,20134,17003,17100,17131,17125,17037,17149,17017,16944,16864
3,3762,3727,3645,3232,3163,3296,3210,3161,3269,3073,3150,3155
4,4186,4094,4113,3475,3484,3579,3578,3401,3505,3617,3522,3394
5,361,356,375,341,318,281,301,320,277,311,268,318
6,4927,4828,5028,4253,4292,4265,4320,4173,4286,4331,4378,4213
7,21104,21153,21024,18065,18007,18172,18217,17868,18024,18188,17792,17839
8,5471,5268,5460,4543,4474,4671,4499,4673,4466,4551,4506,4556
9,8027,7990,7847,6776,6736,6773,6865,6728,6638,6849,6852,6724


In [60]:
ndm.SensorsTrueEnergy(sensor='sipm')

,0,1,2,3,4,5,6,7,8,9,...,1782,1783,1784,1785,1786,1787,1788,1789,1790,1791
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
ndm.h5f.close()

In [40]:
ndm = NDM(path_to_file='../../data/Waveforms/25ns/WF_Na_1k_RWF.h5', access_mode ="r+") 

In [41]:
ndm.Sensors(sensor_table='pmt_t')

NameError: name 'pmt_t' is not defined

In [27]:
ndm = NDM(path_to_file='../../data/Waveforms/25ns/WF_Na_1k_RWF.h5', access_mode ="r+") 

In [28]:
ndm.Geometry()

xdet_min   -198.0
xdet_max    198.0
ydet_min   -198.0
ydet_max    198.0
zdet_min      0.0
zdet_max    532.0
R           198.0
dtype: float64

In [29]:
f,c =ndm.FEE()

In [30]:
f

offset                                                           0
pmt_gain                                                   4.5e+06
V_gain                                                 4.00544e-14
R                                                      3.76512e-13
time_step        [3.76347e+16, 3.88563e+16, 3.78681e+16, 3.8832...
time_daq                                                         1
freq_LPF                                                        25
freq_HPF                                                     0.003
LSB                                                    1.09235e-05
volts_to_adc                                           4.88281e-10
noise_fee_rms                                          0.000390625
noise_adc                                                    7e-10
dtype: object

In [31]:
c/nF

C12    [6.02975, 6.22547, 6.06713, 6.22159, 6.3, 6.09...
dtype: object

In [23]:
ndm = NDM(path_to_file='../../data/Waveforms/25ns/WF_Na_1k_RWF.h5', access_mode ="r+") 

In [24]:
ndm.GeometryData()

xdet_min   -198.0
xdet_max    198.0
ydet_min   -198.0
ydet_max    198.0
zdet_min      0.0
zdet_max    532.0
R           198.0
dtype: float64

In [21]:
ndm = NDM(path_to_file='../../data/Waveforms/25ns/WF_Na_1k_RWF.h5', access_mode ="r+") 

NoSuchNodeError: group ``/RD`` does not have a child named ``sipm``

In [19]:
ndm = NDM(path_to_file='../../data/Waveforms/25ns/WF_Na_1k_RWF.h5', access_mode ="r+") 

AttributeError: 'File' object has no attribute 'RD'

In [15]:
ndm = NDM(path_to_file='../../data/Waveforms/25ns/WF_Na_1k_RWF.h5', access_mode ="r+") 

In [16]:
ndm.h5f

File(filename=../../data/Waveforms/25ns/WF_Na_1k_RWF.h5, title='', mode='r', root_uep='/', filters=Filters(complevel=9, complib='blosc', shuffle=True, fletcher32=False, least_significant_digit=None))
/ (RootGroup) ''
/Detector (Group) ''
/Detector/DetectorGeometry (Table(1,)) ''
  description := {
  "x_det": Float64Col(shape=(2,), dflt=0.0, pos=0),
  "y_det": Float64Col(shape=(2,), dflt=0.0, pos=1),
  "z_det": Float64Col(shape=(2,), dflt=0.0, pos=2),
  "r_det": Float64Col(shape=(), dflt=0.0, pos=3)}
  byteorder := 'little'
  chunkshape := (1170,)
/MC (Group) ''
/MC/FEE (Table(1,)) 'EP-FEE parameters'
  description := {
  "offset": Int16Col(shape=(), dflt=0, pos=0),
  "pmt_gain": Float32Col(shape=(), dflt=0.0, pos=1),
  "V_gain": Float32Col(shape=(), dflt=0.0, pos=2),
  "R": Float32Col(shape=(), dflt=0.0, pos=3),
  "C12": Float32Col(shape=(12,), dflt=0.0, pos=4),
  "CO12": Float32Col(shape=(12,), dflt=0.0, pos=5),
  "time_step": Float32Col(shape=(), dflt=0.0, pos=6),
  "time_daq": Float

In [13]:
ndm = NDM(path_to_file='../../data/Waveforms/25ns/WF_Na_1k_RWF.h5', access_mode ="r+") 

NameError: global name 'path' is not defined